In [34]:
import sys
import time
import numpy as np

import torch
import torch.nn as nn


class RewardTracker:
    def __init__(self, writer, stop_reward):
        self.writer = writer
        self.stop_reward = stop_reward

    def __enter__(self):
        self.ts = time.time()
        self.ts_frame = 0
        self.total_rewards = []
        return self

    def __exit__(self, *args):
        self.writer.close()

    def reward(self, reward, frame, epsilon=None):
        self.total_rewards.append(reward)
        speed = (frame - self.ts_frame) / (time.time() - self.ts)
        self.ts_frame = frame
        self.ts = time.time()
        mean_reward = np.mean(self.total_rewards[-100:])
        epsilon_str = "" if epsilon is None else ", eps %.2f" % epsilon
        print("%d: done %d games, mean reward %.3f, speed %.2f f/s%s" % (
            frame, len(self.total_rewards), mean_reward, speed, epsilon_str
        ))
        sys.stdout.flush()
        if epsilon is not None:
            self.writer.add_scalar("epsilon", epsilon, frame)
        self.writer.add_scalar("speed", speed, frame)
        self.writer.add_scalar("reward_100", mean_reward, frame)
        self.writer.add_scalar("reward", reward, frame)
        if mean_reward > self.stop_reward:
            print("Solved in %d frames!" % frame)
            return True
        return False


class AtariPGN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(AtariPGN, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        fx = x.float() / 256
        conv_out = self.conv(fx).view(fx.size()[0], -1)
        return self.fc(conv_out)

In [35]:
#!/usr/bin/env python3
import gym
import ptan
import time
import random
import numpy as np
import argparse
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.nn.utils as nn_utils
import torch.nn.functional as F
import torch.optim as optim

#from lib import common

GAMMA = 0.99
LEARNING_RATE = 5e-4
ENTROPY_BETA = 0.01
NUM_ENVS = 16

REWARD_STEPS = 4
CLIP_GRAD = 0.1
IMG_SHAPE = (4, 84, 84)


class AtariA2C(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(AtariA2C, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        conv_out_size = self._get_conv_out(input_shape)
        self.policy = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

        self.value = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        fx = x.float() / 256
        conv_out = self.conv(fx).view(fx.size()[0], -1)
        return self.policy(conv_out), self.value(conv_out)


def discount_with_dones(rewards, dones, gamma):
    discounted = []
    r = 0
    for reward, done in zip(rewards[::-1], dones[::-1]):
        r = reward + gamma*r*(1.-done)
        discounted.append(r)
    return discounted[::-1]


def iterate_batches(envs, net, device="cpu"):
    n_actions = envs[0].action_space.n
    act_selector = ptan.actions.ProbabilityActionSelector()
    obs = [e.reset() for e in envs]
    batch_dones = [[False] for _ in range(NUM_ENVS)]
    total_reward = [0.0] * NUM_ENVS
    total_steps = [0] * NUM_ENVS
    mb_obs = np.zeros((NUM_ENVS, REWARD_STEPS) + IMG_SHAPE, dtype=np.uint8)
    mb_rewards = np.zeros((NUM_ENVS, REWARD_STEPS), dtype=np.float32)
    mb_values = np.zeros((NUM_ENVS, REWARD_STEPS), dtype=np.float32)
    mb_actions = np.zeros((NUM_ENVS, REWARD_STEPS), dtype=np.int32)
    mb_probs = np.zeros((NUM_ENVS, REWARD_STEPS, n_actions), dtype=np.float32)

    while True:
        print("batching here")
        batch_dones = [[dones[-1]] for dones in batch_dones]
        done_rewards = []
        done_steps = []
        for n in range(REWARD_STEPS):
            obs_v = ptan.agent.default_states_preprocessor(obs).to(device)
            mb_obs[:, n] = obs_v.data.cpu().numpy()
            logits_v, values_v = net(obs_v)
            probs_v = F.softmax(logits_v, dim=1)
            probs = probs_v.data.cpu().numpy()
            actions = act_selector(probs)
            mb_probs[:, n] = probs
            mb_actions[:, n] = actions
            mb_values[:, n] = values_v.squeeze().data.cpu().numpy()
            for e_idx, e in enumerate(envs):
                o, r, done, _ = e.step(actions[e_idx])
                total_reward[e_idx] += r
                total_steps[e_idx] += 1
                if done:
                    o = e.reset()
                    done_rewards.append(total_reward[e_idx])
                    done_steps.append(total_steps[e_idx])
                    total_reward[e_idx] = 0.0
                    total_steps[e_idx] = 0
                obs[e_idx] = o
                mb_rewards[e_idx, n] = r
                batch_dones[e_idx].append(done)
        # obtain values for the last observation
        obs_v = ptan.agent.default_states_preprocessor(obs).to(device)
        _, values_v = net(obs_v)
        values_last = values_v.squeeze().data.cpu().numpy()

        for e_idx, (rewards, dones, value) in enumerate(zip(mb_rewards, batch_dones, values_last)):
            rewards = rewards.tolist()
            if not dones[-1]:
                rewards = discount_with_dones(rewards + [value], dones[1:] + [False], GAMMA)[:-1]
            else:
                rewards = discount_with_dones(rewards, dones[1:], GAMMA)
            mb_rewards[e_idx] = rewards

        out_mb_obs = mb_obs.reshape((-1,) + IMG_SHAPE)
        out_mb_rewards = mb_rewards.flatten()
        out_mb_actions = mb_actions.flatten()
        out_mb_values = mb_values.flatten()
        out_mb_probs = mb_probs.flatten()
        yield out_mb_obs, out_mb_rewards, out_mb_actions, out_mb_values, out_mb_probs, \
              np.array(done_rewards), np.array(done_steps)


def train_a2c(net, mb_obs, mb_rewards, mb_actions, mb_values, optimizer, tb_tracker, step_idx, device="cpu"):
    print("training here")
    optimizer.zero_grad()
    mb_adv = mb_rewards - mb_values

    adv_v = torch.FloatTensor(mb_adv).to(device)
    obs_v = torch.FloatTensor(mb_obs).to(device)
    rewards_v = torch.FloatTensor(mb_rewards).to(device)
    actions_t = torch.LongTensor(mb_actions).to(device)
    logits_v, values_v = net(obs_v)

    loss_value_v = F.mse_loss(values_v.squeeze(-1), rewards_v)
    print('loss_value_v', loss_value_v.dtype, loss_value_v.shape, loss_value_v)
    
    log_prob_v = F.log_softmax(logits_v, dim=1)
    print('log_prob_v', log_prob_v.dtype, log_prob_v.shape, log_prob_v)
    log_prob_actions_v = adv_v * log_prob_v[range(len(mb_actions)), actions_t]
    print('log_prob_actions_v', log_prob_actions_v.dtype, log_prob_actions_v.shape, log_prob_actions_v)
    loss_policy_v = -log_prob_actions_v.mean()
    print('loss_policy_v', loss_policy_v.dtype, loss_policy_v.shape, loss_policy_v)

    prob_v = F.softmax(logits_v, dim=1)
    print('prob_v', prob_v.dtype, prob_v.shape, prob_v)
    entropy_loss_v = (prob_v * log_prob_v).sum(dim=1).mean()
    print('entropy_loss_v', entropy_loss_v.dtype, entropy_loss_v.shape, entropy_loss_v)
    loss_v = ENTROPY_BETA * entropy_loss_v + loss_value_v + loss_policy_v
    print('loss_v', loss_v.dtype, loss_v.shape, loss_v)
    loss_v.backward()
    nn_utils.clip_grad_norm_(net.parameters(), CLIP_GRAD)
    optimizer.step()

    return obs_v


def set_seed(seed, envs=None, cuda=False):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed(seed)

    if envs:
        for idx, env in enumerate(envs):
            env.seed(seed + idx)


if __name__ == "__main__":

    device = torch.device("cpu")

    make_env = lambda: ptan.common.wrappers.wrap_dqn(gym.make("BreakoutNoFrameskip-v4"))
    envs = [make_env() for _ in range(NUM_ENVS)]
    writer = SummaryWriter(comment="-pong-a2c-r2_")
    set_seed(20, envs)

    net = AtariA2C(envs[0].observation_space.shape, envs[0].action_space.n).to(device)
    #print(net)

    optimizer = optim.RMSprop(net.parameters(), lr=LEARNING_RATE, eps=1e-5)

    step_idx = 0
    total_steps = 0
    best_reward = None
    ts_start = time.time()

    with RewardTracker(writer, stop_reward=18) as tracker:
        with ptan.common.utils.TBMeanTracker(writer, batch_size=10) as tb_tracker:
            for mb_obs, mb_rewards, mb_actions, mb_values, _, done_rewards, done_steps in iterate_batches(envs, net, device=device):
                if len(done_rewards) > 0:
                    total_steps += sum(done_steps)
                    speed = total_steps / (time.time() - ts_start)
                    if best_reward is None:
                        best_reward = done_rewards.max()
                    elif best_reward < done_rewards.max():
                        best_reward = done_rewards.max()
                    tb_tracker.track("total_reward_max", best_reward, step_idx)
                    tb_tracker.track("total_reward", done_rewards, step_idx)
                    tb_tracker.track("total_steps", done_steps, step_idx)
                    print("%d: done %d episodes, mean_reward=%.2f, best_reward=%.2f, speed=%.2f" % (
                        step_idx, len(done_rewards), done_rewards.mean(), best_reward, speed))

                train_a2c(net, mb_obs, mb_rewards, mb_actions, mb_values,
                          optimizer, tb_tracker, step_idx, device=device)
                step_idx += 1

batching here
training here
loss_value_v torch.float32 torch.Size([]) tensor(1.2038e-06, grad_fn=<MseLossBackward>)
log_prob_v torch.float32 torch.Size([64, 4]) tensor([[-1.3535, -1.3975, -1.4256, -1.3701],
        [-1.3534, -1.3976, -1.4256, -1.3702],
        [-1.3535, -1.3972, -1.4257, -1.3702],
        [-1.3535, -1.3975, -1.4255, -1.3702],
        [-1.3537, -1.3974, -1.4256, -1.3699],
        [-1.3535, -1.3975, -1.4256, -1.3701],
        [-1.3536, -1.3972, -1.4258, -1.3701],
        [-1.3535, -1.3972, -1.4258, -1.3701],
        [-1.3535, -1.3975, -1.4256, -1.3701],
        [-1.3534, -1.3976, -1.4256, -1.3702],
        [-1.3535, -1.3972, -1.4257, -1.3702],
        [-1.3536, -1.3975, -1.4255, -1.3701],
        [-1.3537, -1.3974, -1.4256, -1.3699],
        [-1.3537, -1.3975, -1.4257, -1.3699],
        [-1.3536, -1.3974, -1.4256, -1.3701],
        [-1.3537, -1.3974, -1.4258, -1.3699],
        [-1.3537, -1.3974, -1.4256, -1.3699],
        [-1.3535, -1.3974, -1.4257, -1.3700],
        [-1

batching here
training here
loss_value_v torch.float32 torch.Size([]) tensor(3.1505e-07, grad_fn=<MseLossBackward>)
log_prob_v torch.float32 torch.Size([64, 4]) tensor([[-1.3606, -1.3925, -1.4202, -1.3729],
        [-1.3605, -1.3924, -1.4202, -1.3731],
        [-1.3606, -1.3924, -1.4201, -1.3730],
        [-1.3606, -1.3924, -1.4201, -1.3731],
        [-1.3607, -1.3923, -1.4201, -1.3731],
        [-1.3607, -1.3925, -1.4200, -1.3730],
        [-1.3605, -1.3927, -1.4201, -1.3728],
        [-1.3606, -1.3924, -1.4202, -1.3731],
        [-1.3607, -1.3924, -1.4201, -1.3729],
        [-1.3607, -1.3924, -1.4201, -1.3730],
        [-1.3607, -1.3924, -1.4201, -1.3730],
        [-1.3604, -1.3926, -1.4201, -1.3731],
        [-1.3604, -1.3927, -1.4203, -1.3728],
        [-1.3607, -1.3923, -1.4205, -1.3728],
        [-1.3607, -1.3922, -1.4204, -1.3730],
        [-1.3603, -1.3925, -1.4204, -1.3730],
        [-1.3606, -1.3925, -1.4201, -1.3729],
        [-1.3606, -1.3925, -1.4200, -1.3731],
        [-1

training here
loss_value_v torch.float32 torch.Size([]) tensor(1.4474e-08, grad_fn=<MseLossBackward>)
log_prob_v torch.float32 torch.Size([64, 4]) tensor([[-1.3661, -1.3930, -1.4146, -1.3721],
        [-1.3661, -1.3931, -1.4146, -1.3722],
        [-1.3661, -1.3930, -1.4146, -1.3721],
        [-1.3661, -1.3930, -1.4146, -1.3722],
        [-1.3660, -1.3932, -1.4146, -1.3721],
        [-1.3661, -1.3932, -1.4144, -1.3722],
        [-1.3661, -1.3930, -1.4146, -1.3722],
        [-1.3661, -1.3930, -1.4145, -1.3723],
        [-1.3660, -1.3930, -1.4145, -1.3723],
        [-1.3662, -1.3931, -1.4146, -1.3721],
        [-1.3661, -1.3930, -1.4147, -1.3722],
        [-1.3661, -1.3930, -1.4146, -1.3722],
        [-1.3661, -1.3933, -1.4145, -1.3720],
        [-1.3661, -1.3931, -1.4145, -1.3722],
        [-1.3661, -1.3930, -1.4145, -1.3723],
        [-1.3661, -1.3931, -1.4145, -1.3723],
        [-1.3660, -1.3932, -1.4145, -1.3721],
        [-1.3661, -1.3933, -1.4145, -1.3721],
        [-1.3661, -1.3930

training here
loss_value_v torch.float32 torch.Size([]) tensor(1.3824e-06, grad_fn=<MseLossBackward>)
log_prob_v torch.float32 torch.Size([64, 4]) tensor([[-1.3666, -1.3901, -1.4151, -1.3741],
        [-1.3667, -1.3898, -1.4152, -1.3742],
        [-1.3666, -1.3900, -1.4151, -1.3741],
        [-1.3669, -1.3896, -1.4153, -1.3741],
        [-1.3668, -1.3901, -1.4151, -1.3738],
        [-1.3669, -1.3899, -1.4150, -1.3740],
        [-1.3669, -1.3898, -1.4153, -1.3739],
        [-1.3670, -1.3898, -1.4151, -1.3739],
        [-1.3668, -1.3898, -1.4152, -1.3740],
        [-1.3668, -1.3895, -1.4156, -1.3740],
        [-1.3666, -1.3899, -1.4152, -1.3741],
        [-1.3668, -1.3899, -1.4153, -1.3739],
        [-1.3665, -1.3900, -1.4152, -1.3741],
        [-1.3667, -1.3897, -1.4153, -1.3742],
        [-1.3666, -1.3900, -1.4152, -1.3741],
        [-1.3669, -1.3896, -1.4153, -1.3741],
        [-1.3665, -1.3901, -1.4151, -1.3741],
        [-1.3667, -1.3898, -1.4152, -1.3742],
        [-1.3667, -1.3899

training here
loss_value_v torch.float32 torch.Size([]) tensor(0.0007, grad_fn=<MseLossBackward>)
log_prob_v torch.float32 torch.Size([64, 4]) tensor([[-1.2955, -1.2911, -1.3609, -1.6356],
        [-1.2954, -1.2910, -1.3612, -1.6355],
        [-1.2957, -1.2911, -1.3610, -1.6352],
        [-1.2954, -1.2910, -1.3611, -1.6355],
        [-1.2955, -1.2912, -1.3611, -1.6351],
        [-1.2955, -1.2912, -1.3610, -1.6353],
        [-1.2954, -1.2912, -1.3612, -1.6352],
        [-1.2956, -1.2910, -1.3608, -1.6357],
        [-1.2955, -1.2911, -1.3610, -1.6354],
        [-1.2956, -1.2913, -1.3607, -1.6354],
        [-1.2955, -1.2909, -1.3610, -1.6356],
        [-1.2956, -1.2911, -1.3610, -1.6354],
        [-1.2954, -1.2914, -1.3608, -1.6355],
        [-1.2953, -1.2912, -1.3610, -1.6356],
        [-1.2954, -1.2914, -1.3609, -1.6353],
        [-1.2954, -1.2910, -1.3612, -1.6354],
        [-1.2957, -1.2912, -1.3607, -1.6355],
        [-1.2956, -1.2909, -1.3610, -1.6355],
        [-1.2957, -1.2912, -1

...

94: done 2 episodes, mean_reward=0.00, best_reward=2.00, speed=232.60
training here
loss_value_v torch.float32 torch.Size([]) tensor(0.0006, grad_fn=<MseLossBackward>)
log_prob_v torch.float32 torch.Size([64, 4]) tensor([[-1.4115, -1.2142, -1.3803, -1.5713],
        [-1.4115, -1.2139, -1.3803, -1.5716],
        [-1.4115, -1.2141, -1.3803, -1.5715],
        [-1.4115, -1.2139, -1.3803, -1.5716],
        [-1.4115, -1.2140, -1.3804, -1.5716],
        [-1.4115, -1.2140, -1.3803, -1.5716],
        [-1.4114, -1.2141, -1.3804, -1.5714],
        [-1.4114, -1.2140, -1.3804, -1.5717],
        [-1.4115, -1.2140, -1.3803, -1.5716],
        [-1.4116, -1.2138, -1.3803, -1.5717],
        [-1.4115, -1.2142, -1.3804, -1.5712],
        [-1.4115, -1.2140, -1.3803, -1.5715],
        [-1.4115, -1.2141, -1.3803, -1.5715],
        [-1.4115, -1.2141, -1.3803, -1.5715],
        [-1.4115, -1.2141, -1.3803, -1.5715],
        [-1.4115, -1.2141, -1.3803, -1.5715],
        [-1.4115, -1.2140, -1.3803, -1.5716],
     

training here
loss_value_v torch.float32 torch.Size([]) tensor(7.8272e-06, grad_fn=<MseLossBackward>)
log_prob_v torch.float32 torch.Size([64, 4]) tensor([[-1.4032, -1.1910, -1.3916, -1.6014],
        [-1.4032, -1.1910, -1.3916, -1.6014],
        [-1.4032, -1.1910, -1.3916, -1.6014],
        [-1.4032, -1.1910, -1.3916, -1.6014],
        [-1.4032, -1.1908, -1.3916, -1.6018],
        [-1.4032, -1.1907, -1.3916, -1.6018],
        [-1.4032, -1.1909, -1.3916, -1.6015],
        [-1.4032, -1.1911, -1.3916, -1.6013],
        [-1.4034, -1.1905, -1.3916, -1.6020],
        [-1.4033, -1.1907, -1.3916, -1.6018],
        [-1.4033, -1.1911, -1.3916, -1.6011],
        [-1.4033, -1.1908, -1.3916, -1.6016],
        [-1.4032, -1.1911, -1.3916, -1.6013],
        [-1.4033, -1.1909, -1.3916, -1.6015],
        [-1.4032, -1.1910, -1.3916, -1.6014],
        [-1.4033, -1.1909, -1.3916, -1.6016],
        [-1.4032, -1.1910, -1.3916, -1.6014],
        [-1.4032, -1.1909, -1.3917, -1.6015],
        [-1.4032, -1.1909

98: done 2 episodes, mean_reward=0.00, best_reward=2.00, speed=228.33
training here
loss_value_v torch.float32 torch.Size([]) tensor(0.0314, grad_fn=<MseLossBackward>)
log_prob_v torch.float32 torch.Size([64, 4]) tensor([[-1.4023, -1.1950, -1.3892, -1.5995],
        [-1.4023, -1.1951, -1.3892, -1.5994],
        [-1.4022, -1.1950, -1.3893, -1.5994],
        [-1.4023, -1.1950, -1.3892, -1.5994],
        [-1.4022, -1.1951, -1.3893, -1.5994],
        [-1.4022, -1.1949, -1.3893, -1.5997],
        [-1.4022, -1.1950, -1.3892, -1.5996],
        [-1.4022, -1.1950, -1.3893, -1.5995],
        [-1.4023, -1.1949, -1.3893, -1.5996],
        [-1.4023, -1.1949, -1.3893, -1.5996],
        [-1.4023, -1.1947, -1.3893, -1.5998],
        [-1.4023, -1.1948, -1.3893, -1.5997],
        [-1.4022, -1.1956, -1.3893, -1.5987],
        [-1.4023, -1.1950, -1.3893, -1.5995],
        [-1.4023, -1.1952, -1.3892, -1.5993],
        [-1.4023, -1.1951, -1.3892, -1.5994],
        [-1.4023, -1.1951, -1.3892, -1.5993],
     

100: done 2 episodes, mean_reward=0.00, best_reward=2.00, speed=226.43
training here
loss_value_v torch.float32 torch.Size([]) tensor(0.1075, grad_fn=<MseLossBackward>)
log_prob_v torch.float32 torch.Size([64, 4]) tensor([[-1.4444, -1.0773, -1.4226, -1.7011],
        [-1.4444, -1.0774, -1.4226, -1.7009],
        [-1.4443, -1.0774, -1.4226, -1.7009],
        [-1.4443, -1.0774, -1.4226, -1.7009],
        [-1.4443, -1.0774, -1.4226, -1.7009],
        [-1.4444, -1.0772, -1.4226, -1.7011],
        [-1.4443, -1.0775, -1.4226, -1.7007],
        [-1.4444, -1.0772, -1.4226, -1.7011],
        [-1.4445, -1.0771, -1.4226, -1.7011],
        [-1.4446, -1.0769, -1.4227, -1.7014],
        [-1.4446, -1.0770, -1.4226, -1.7012],
        [-1.4444, -1.0773, -1.4226, -1.7010],
        [-1.4444, -1.0774, -1.4226, -1.7008],
        [-1.4443, -1.0774, -1.4226, -1.7008],
        [-1.4443, -1.0774, -1.4226, -1.7008],
        [-1.4443, -1.0774, -1.4226, -1.7008],
        [-1.4444, -1.0775, -1.4225, -1.7008],
    

KeyboardInterrupt: 

In [ ]:
mb_actions = np.zeros((6, 3), dtype=np.int32)

In [ ]:
mb_actions

In [ ]:
device = torch.device("cpu")

In [49]:
test = -36.6

In [50]:
test1 = torch.FloatTensor(test).to(device)

TypeError: new(): data must be a sequence (got float)

In [48]:
test1

tensor([0.1784, 0.1803, 0.1821, 0.1839, 0.1785, 0.1803, 0.1821, 0.1839, 0.1786,
        0.1804, 0.1823, 0.1841, 0.1783, 0.1801, 0.1819, 0.1837, 0.1783, 0.1801,
        0.1820, 0.1838, 1.1684, 1.1802, 0.1820, 0.1838, 0.1786, 0.1804, 0.1822,
        0.1840, 0.1784, 0.1802, 0.1820, 0.1838, 0.1784, 0.1802, 0.1820, 0.1838,
        0.1783, 0.1801, 0.1820, 0.1838, 0.1787, 0.1805, 0.1823, 0.1842, 0.1783,
        0.1801, 0.1819, 0.1837, 0.1783, 0.1801, 0.1819, 0.1838, 0.1782, 0.1800,
        0.1818, 0.1836, 0.1784, 0.1802, 0.1820, 0.1839, 0.1784, 0.1802, 0.1821,
        0.1839])

In [39]:
adv_v = torch.tensor(test).to(device)

In [40]:
adv_v.dtype

torch.float32

In [41]:
adv_v.long()

tensor(-36)

In [44]:
test2 = test1 + 35
test2

tensor(-1.6000)